<a href="https://colab.research.google.com/github/YasirHabib/Deep-Learning-Convolutional-Neural-Networks-in-Python/blob/master/svhn_th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

In [0]:
import theano.tensor as T
import theano

In [0]:
class HiddenLayer():
  def __init__(self, D, M):
    W = np.random.randn(D, M) / np.sqrt(D)
    b = np.zeros(M)
    
    self.W = theano.shared(W)
    self.b = theano.shared(b)
    
    self.params = [self.W, self.b]
    
  def HiddenLayer_forward(self, X):
    return T.nnet.relu(X.dot(self.W)+self.b)

In [0]:
class FinalLayer():
  def __init__(self, M, K):
    W = np.random.randn(M, K) / np.sqrt(M)
    b = np.zeros(K)
    
    self.W = theano.shared(W)
    self.b = theano.shared(b)
    
    self.params = [self.W, self.b]
    
  def FinalLayer_forward(self, Z):
    return T.nnet.softmax(Z.dot(self.W)+self.b)

In [61]:
class ANN():
  def __init__(self, size):
    self.size = size
    
  def fit(self, Xtrain, Ytrain, Xtest, Ytest, lr=10e-2, reg=0.01, training_epochs=10, batch_sz=500):
    N = Xtrain.shape[0]    
    K = len(set(Ytrain))
    poolsz = (2, 2)
    
    # after conv will be of dimension 32 - 5 + 1 = 28
    # after downsample 28 / 2 = 14
    W1_shape = (20, 3, 5, 5) # (num_feature_maps, num_color_channels, filter_width, filter_height)
    W1 = self.init_filter(W1_shape)
    b1 = np.zeros(W1_shape[0])
    
    # after conv will be of dimension 14 - 5 + 1 = 10
    # after downsample 10 / 2 = 5
    W2_shape = (50, 20, 5, 5) # (num_feature_maps, num_color_channels, filter_width, filter_height)
    W2 = self.init_filter(W2_shape)
    b2 = np.zeros(W2_shape[0])
    
    self.layers = []
    h = HiddenLayer(2, self.size)
    self.layers.append(h)
    
    f = FinalLayer(self.size, K)
    self.layers.append(f)
    
    self.parameters = []
    for obj in self.layers:
      self.parameters += obj.params
      
    X = T.fmatrix('X')
    Y = T.ivector('Y')
      
    pY = self.th_forward(X)
    
    rcost = reg*T.sum([(p*p).sum() for p in self.parameters])
    cost = -T.mean(T.log(pY[T.arange(Y.shape[0]), Y])) + rcost
  
  def init_filter(self, shape):
    return np.random.randn(*shape) * np.sqrt(2.0 / np.prod(shape[1:]))
    
  def convpool(self, ):
    
    
  def th_forward(self, X):
    for obj in self.layers[:-1]:
      Z = obj.HiddenLayer_forward(X)
    for obj in self.layers[-1:]:
      pY = obj.FinalLayer_forward(Z)
    

IndentationError: ignored

In [0]:
def main():
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount = True)
  
  train = scipy.io.loadmat('/content/gdrive/My Drive/Colab Notebooks/Convolutional Neural Networks/train_32x32.mat')
  test = scipy.io.loadmat('/content/gdrive/My Drive/Colab Notebooks/Convolutional Neural Networks/test_32x32.mat')
  
  Xtrain = train['X']             # 'X' is a dictionary key pointing to the input data
  Ytrain = train['y']             # 'y' is a dictionary key pointing to the targets
  #print(Xtrain.shape)            # (32, 32, 3, 73257). This means each N (73257 here) is 32x32x3. H=32, W=32, Colors=3
  #print(Ytrain.shape)            # (73257, 1)
  
  Ytrain = Ytrain.flatten() - 1
  #print(Ytrain.shape)            # (73257,)
  
  Xtest = test['X']
  Ytest = test['y']
  #print(Xtest.shape)             # (32, 32, 3, 26032). This means each N (26032 here) is 32x32x3. H=32, W=32, Colors=3
  #print(Ytest.shape)             # (26032, 1)
  
  Ytest = Ytest.flatten() - 1
  #print(Ytest.shape)             # (26032,)
  
  Xtrain = (Xtrain.transpose(3,2,0,1) / 255).astype(np.float32)
  Xtest = (Xtest.transpose(3,2,0,1) / 255).astype(np.float32)
  
  model = ANN(500)
  model.fit(Xtrain, Ytrain, Xtest, Ytest)
  
if __name__ == '__main__':
  main()